In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm

n_steps = 300
n_nodes = 10
tx_prob = 1/n_nodes
data_tx = np.zeros((n_steps, n_nodes))
data_aoi = np.zeros((n_steps, n_nodes))
reward = np.zeros((n_steps, 1))
data_tx[0,:] = 0
data_aoi[0,:] = 0

AGECOEFF = 0.1

def get_reward():
    for i in range(1, n_steps):
        data_tx[i] = (np.random.rand(n_nodes)<tx_prob).astype(int)
        # Success if only one node transmits
        if np.sum(data_tx[i]) == 1:
            succ_idx = np.where(data_tx[i])[0]
            for j in range(n_nodes):
                if j == succ_idx:
                    data_aoi[i,j] = 0
                else:
                    data_aoi[i,j] = data_aoi[i-1,j] + (1/n_steps)
            reward[i] = 1 - AGECOEFF*max(data_aoi[i,:]) # 보낸 갯수만큼 보상을 준다.
        else:
            succ_idx = None
            data_aoi[i,:] = data_aoi[i-1,:] + (1/n_steps)
            reward[i] = 0 - AGECOEFF*max(data_aoi[i,:]) # 보낸 갯수만큼 보상을 준다.
    return sum(reward)

reward_avg = 0
for _ in tqdm(range(1000)):
    reward_avg += get_reward()
print(reward_avg/1000)

100%|██████████| 1000/1000 [00:04<00:00, 218.16it/s]

[109.20425167]


In [9]:
df_sim = pd.read_csv('outputs/R2inforce_n10_density1_max_episode_length300/log_Feb22_03-16-44.csv')
df_sim = df_sim.loc[df_sim["episode"] == 4999]

In [13]:
df_sim.iloc[:, 2:12]

,action_0,action_1,action_2,action_3,action_4,action_5,action_6,action_7,action_8,action_9
1499700,0,1,0,0,0,0,1,0,0,0
1499701,0,0,0,0,0,0,1,0,0,0
1499702,0,0,0,0,0,0,1,0,0,0
1499703,0,0,0,0,1,0,1,1,0,0
1499704,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1499995,0,0,0,0,0,0,1,0,0,1
1499996,0,0,0,0,0,0,0,0,0,0
1499997,0,0,0,1,0,0,1,0,0,0
1499998,0,0,0,0,0,0,1,0,0,0


In [17]:
df_sim["success_indicator"] = (df_sim.iloc[:, 2:12].sum(axis=1)==1).astype(int)
df_sim["reward"] = df_sim["success_indicator"] - AGECOEFF*df_sim.iloc[:, 12:22].max(axis=1)
print(df_sim["reward"].sum())

183.95000000000005
